In [2]:
import pandas as pd
import numpy as np
import urllib.request
import bz2
import re

In [3]:
# Currently there are some games that have no moves and do not follow the moveNums Regex pattern. Unsure how to fix this eloquently

with urllib.request.urlopen('https://database.lichess.org/standard/lichess_db_standard_rated_2014-09.pgn.bz2') as f:
    decompressed = bz2.BZ2File(f, 'r')
    bString = decompressed.read()
    # find the events strings and pull the game type
    events = np.array([x.split(b"\"")[1].split(b" ")[1].decode('UTF-8') for x in re.findall(b"\[Event.*\]", bString)])
    # find the results string and pull the result only
    results = np.array([x.split(b"\"")[1].decode('UTF-8') for x in re.findall(b"\[Result.*\]", bString)])
    # find white's elo and pull it as an int (if ? then we set it to 0)
    whiteELOs = np.array([int(x.split(b"\"")[1]) if x.split(b"\"")[1] != b"?" else 0 for x in re.findall(b"\[WhiteElo.*\]", bString)])
    # find black's elo and pull it as an int (if ? then we set it to 0)
    blackELOs = np.array([int(x.split(b"\"")[1]) if x.split(b"\"")[1] != b"?" else 0 for x in re.findall(b"\[BlackElo.*\]", bString)])
    # find the time control string and pull the game style
    times = np.array([x.split(b"\"")[1].decode('UTF-8') for x in re.findall(b"\[TimeControl.*\]", bString)])
    # find the termination type of the game
    terminations = np.array([x.split(b"\"")[1].decode('UTF-8') for x in re.findall(b"\[Termination.*\]", bString)])
    # find the last move recorded in the move list and pull it as an int
    moveNums = np.array([(0 if len(re.findall(b"\d+\.", x[0])) == 0 else int(re.findall(b"\d+\.", x[0])[-1][:-1])) for x in re.findall(b"(\]\\n\\n.*?(0-1|1-0|1/2-1/2))", bString)])
    
    all_data = np.vstack((events, results, whiteELOs, blackELOs, times, terminations, moveNums)).T
    df = pd.DataFrame(all_data, columns=['Game Type', 'Result', 'White ELO', 'Black ELO', 'Time Control', 'Termination', 'Moves'])
    df.to_pickle('./Chess_Data.pickle')


Let's check to make sure these grabbed the values we want, starting with the original PGN and then the values grabbed.

In [12]:
print(bString[:743].decode('UTF-8'))

print('-'*130)

print(df.loc[0,:])


[Event "Rated Bullet game"]
[Site "https://lichess.org/1Vimq9SL"]
[White "KACHAL"]
[Black "justplaybi"]
[Result "0-1"]
[UTCDate "2014.08.31"]
[UTCTime "22:00:33"]
[WhiteElo "1833"]
[BlackElo "1823"]
[WhiteRatingDiff "-11"]
[BlackRatingDiff "+12"]
[ECO "A06"]
[Opening "Zukertort Opening: Tennison Gambit"]
[TimeControl "120+0"]
[Termination "Time forfeit"]

1. e4 d5 2. Nf3 dxe4 3. Ne5 Nf6 4. d4 exd3 5. Bxd3 e6 6. Nc3 Bd6 7. Nc4 Nc6 8. Bg5 Be7 9. Qd2 Nb4 10. O-O-O Nxd3+ 11. Qxd3 Qxd3 12. Rxd3 O-O 13. Nb5 c6 14. Nc7 Rb8 15. Rg1 Nd5 16. Bxe7 Nxe7 17. Rg3 Nd5 18. Nxd5 exd5 19. Nd6 Be6 20. Rb3 b6 21. Re1 d4 22. Rg3 c5 23. Ne4 Kh8 24. f4 Bf5 25. Nd6 Be6 26. f5 Rbd8 27. fxe6 Rxd6 28. e7 Re8 29. Rf3 f6 30. Rf4 Kg8 31. Rfe4 Kf7 32. R1e2 Rd7 0-1
----------------------------------------------------------------------------------------------------------------------------------
Game Type             Bullet
Result                   0-1
White ELO               1833
Black ELO               1823
Time Cont